In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [94]:
# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128  # Decrease batch size if you don't have enough memory
display_step = 1

n_features = 784  # MNIST data features (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

n_hidden_layer = 256 # layer number of features

In [108]:
tf.reset_default_graph()

# Store layers weight & bias
w_h = tf.Variable(tf.random_normal([n_features, n_hidden_layer]))
w_out = tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))

b_h = tf.Variable(tf.random_normal([n_hidden_layer]))
b_out = tf.Variable(tf.random_normal([n_classes]))

In [109]:
# tf Graph input
features = tf.placeholder(tf.float32, shape=(None, 28, 28, 1))
labels = tf.placeholder(tf.float32, shape=(None, n_classes))

# this shapes the matrix into a column vector
features_flat = tf.reshape(features, [-1, n_features])

The MNIST data is made up of 28px by 28px images with a single channel. The `tf.reshape()` function above reshapes the 28px by 28px matrices in x into vectors of 784px by 1px.

In [110]:
# the layers
layer_1 = tf.add(tf.matmul(features_flat, w_h), b_h)
layer_1 = tf.nn.relu(layer_1)

# output layer with linear activation
logits = tf.add(tf.matmul(layer_1, w_out), b_out)

In [111]:
# the optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# accuracy calculation
is_correct_prediction = tf.equal(tf.argmax(logits, 1), tf.arg_max(labels, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

In [112]:
# train the network in a session
# save and restore training progress

saver = tf.train.Saver()
save_file = './mnist_model.ckpt'

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # initialize the placeholders
    
    # training epoch
    for epoch in range(training_epochs):
        n_batches = int(mnist.train.num_examples / batch_size)
        
        # batch in the epoch
        for i in range(n_batches):
            batch_features, batch_labels = mnist.train.next_batch(batch_size)
            
            # optimization
            sess.run(optimizer, feed_dict={features: batch_features, labels: batch_labels})
            
        # compute accuracy periodically
        if epoch % 10 == 0:
            validation_accuracy = sess.run(
                accuracy,
                feed_dict={
                    features: mnist.validation.images,
                    labels: mnist.validation.labels})
            
            print('Epoch {}, accuracy: {}'.format(epoch, validation_accuracy))
            
        # save the trained model    
    save_path = saver.save(sess, save_file)

Epoch 0, accuracy: 0.3928000032901764
Epoch 10, accuracy: 0.8026000261306763
Epoch 20, accuracy: 0.8425999879837036
Epoch 30, accuracy: 0.8611999750137329


The model loading does not seem to work.

In [107]:
# restore training progress

# tf.reset_default_graph()

# # Store layers weight & bias
# w_h = tf.Variable(tf.random_normal([n_features, n_hidden_layer]), name='w_h')
# w_out = tf.Variable(tf.random_normal([n_hidden_layer, n_classes]), name='w_out')

# b_h = tf.Variable(tf.random_normal([n_hidden_layer]), name='b_h')
# b_out = tf.Variable(tf.random_normal([n_classes]), name='b_out')


saver = tf.train.Saver()

with tf.Session() as sess:
    # load graph state
    # if you don't recreate the same variables, which can be a pain for large network,
    # then you must import the meta file
#     saver = tf.train.import_meta_graph('mnist_model.ckpt.meta')
#     saver.restore(sess, './mnist_model.ckpt')
    
    test_accuracy = sess.run(
        accuracy,
        feed_dict={
            features: mnist.test.images,
            labels: mnist.test.labels})
    
    print('Test accuracy: {}'.format(test_accuracy))


FailedPreconditionError: Attempting to use uninitialized value b_out
	 [[Node: b_out/read = Identity[T=DT_FLOAT, _class=["loc:@b_out"], _device="/job:localhost/replica:0/task:0/cpu:0"](b_out)]]

Caused by op 'b_out/read', defined at:
  File "/Users/kgraham/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/kgraham/anaconda3/envs/tensorflow/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/kgraham/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/kgraham/anaconda3/envs/tensorflow/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/kgraham/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/Users/kgraham/anaconda3/envs/tensor